# **Lending Club Case Study - DATA WRANGLING**

**Data**

We have been provided with the Private Data of Lending Club.
The complete loan data for all loans issued through the time period 2007 to 2018.

We also have a data dictionary which describes the meaning of these variables.

Let's start with importing all the required libraries for the analysis.

In [6]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import rcParams
from scipy import stats
from pandas_profiling import ProfileReport
sns.set(style='whitegrid')


%matplotlib inline
 
rcParams['figure.figsize'] = 14,6
 
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/lcdatadictionaryxlsx/LCDataDictionary.xlsx
/kaggle/input/lending-club/rejected_2007_to_2018Q4.csv.gz
/kaggle/input/lending-club/accepted_2007_to_2018Q4.csv.gz
/kaggle/input/lending-club/accepted_2007_to_2018q4.csv/accepted_2007_to_2018Q4.csv
/kaggle/input/lending-club/rejected_2007_to_2018q4.csv/rejected_2007_to_2018Q4.csv


Reading the loan data file

In [2]:
df = pd.read_csv('/kaggle/input/lending-club/accepted_2007_to_2018q4.csv/accepted_2007_to_2018Q4.csv', low_memory = False)

In [80]:
pd.options.display.max_rows = 1000
df.head(5)

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,68407277,NaN,3600.0,3600.0,3600.0,36 months,13.99,123.03,C,C4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,68355089,NaN,24700.0,24700.0,24700.0,36 months,11.99,820.28,C,C1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
2,68341763,NaN,20000.0,20000.0,20000.0,60 months,10.78,432.66,B,B4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
3,66310712,NaN,35000.0,35000.0,35000.0,60 months,14.85,829.90,C,C5,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
4,68476807,NaN,10400.0,10400.0,10400.0,60 months,22.45,289.91,F,F1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN


**Data Cleaning**

Checking Data for null values and data types. Since data shape is very large descibe() functionality wasn't applied.**

In [82]:
df.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2260701 entries, 0 to 2260700
Data columns (total 151 columns):
 #   Column                                      Non-Null Count    Dtype  
---  ------                                      --------------    -----  
 0   id                                          2260701 non-null  object 
 1   member_id                                   0 non-null        float64
 2   loan_amnt                                   2260668 non-null  float64
 3   funded_amnt                                 2260668 non-null  float64
 4   funded_amnt_inv                             2260668 non-null  float64
 5   term                                        2260668 non-null  object 
 6   int_rate                                    2260668 non-null  float64
 7   installment                                 2260668 non-null  float64
 8   grade                                       2260668 non-null  object 
 9   sub_grade                                   2260668 non-

Checking for Missing Values Percentage for each feature or columns

In [4]:
missing = round(100*(df.isnull().sum()/len(df.id)), 2)
print(missing.loc[missing>0])

member_id                100.00
emp_title                  7.39
emp_length                 6.50
desc                      94.42
title                      1.03
                          ...  
settlement_status         98.49
settlement_date           98.49
settlement_amount         98.49
settlement_percentage     98.49
settlement_term           98.49
Length: 102, dtype: float64


List of columns with more than 30% data missing

In [21]:
columns_with_missing_values = list(missing[missing > 30].index)
print(columns_with_missing_values)
len(columns_with_missing_values)

['member_id', 'desc', 'mths_since_last_delinq', 'mths_since_last_record', 'next_pymnt_d', 'mths_since_last_major_derog', 'annual_inc_joint', 'dti_joint', 'verification_status_joint', 'open_acc_6m', 'open_act_il', 'open_il_12m', 'open_il_24m', 'mths_since_rcnt_il', 'total_bal_il', 'il_util', 'open_rv_12m', 'open_rv_24m', 'max_bal_bc', 'all_util', 'inq_fi', 'total_cu_tl', 'inq_last_12m', 'mths_since_recent_bc_dlq', 'mths_since_recent_revol_delinq', 'revol_bal_joint', 'sec_app_fico_range_low', 'sec_app_fico_range_high', 'sec_app_earliest_cr_line', 'sec_app_inq_last_6mths', 'sec_app_mort_acc', 'sec_app_open_acc', 'sec_app_revol_util', 'sec_app_open_act_il', 'sec_app_num_rev_accts', 'sec_app_chargeoff_within_12_mths', 'sec_app_collections_12_mths_ex_med', 'sec_app_mths_since_last_major_derog', 'hardship_type', 'hardship_reason', 'hardship_status', 'deferral_term', 'hardship_amount', 'hardship_start_date', 'hardship_end_date', 'payment_plan_start_date', 'hardship_length', 'hardship_dpd', 'ha

58

Feature description for better understanding of features

In [66]:
exc = pd.read_excel('/kaggle/input/lcdatadictionaryxlsx/LCDataDictionary.xlsx').dropna()
pd.options.display.max_colwidth =100
exc.set_index('LoanStatNew', inplace=True)
exc

,Description
LoanStatNew,
acc_now_delinq,The number of accounts on which the borrower is now delinquent.
acc_open_past_24mths,Number of trades opened in past 24 months.
addr_state,The state provided by the borrower in the loan application
all_util,Balance to credit limit on all trades
annual_inc,The self-reported annual income provided by the borrower during registration.
annual_inc_joint,The combined self-reported annual income provided by the co-borrowers during registration
application_type,Indicates whether the loan is an individual application or a joint application with two co-borro...
avg_cur_bal,Average current balance of all accounts
bc_open_to_buy,Total open to buy on revolving bankcards.


There are many columns in the data with 100% missing values. Removing all the columns with more than 30% missing values. There were 58 columns with more than 30% values as missing. Removing such columns as these columns clearly add noise and wont help in analysis.

In [22]:
df1 = df.drop(columns_with_missing_values,axis=1)
df1.shape

(2260701, 93)

After removing such columns 93 columns remain.
Let's examine further for missing values.
Checking percentage of data missing for remaining features.

In [31]:
missing = round(100*(df1.isnull().sum()/len(df1.id)), 2)
missing[missing != 0]

emp_title                      7.39
emp_length                     6.50
title                          1.03
dti                            0.08
revol_util                     0.08
last_pymnt_d                   0.11
collections_12_mths_ex_med     0.01
tot_coll_amt                   3.11
tot_cur_bal                    3.11
total_rev_hi_lim               3.11
acc_open_past_24mths           2.21
avg_cur_bal                    3.11
bc_open_to_buy                 3.32
bc_util                        3.37
chargeoff_within_12_mths       0.01
mo_sin_old_il_acct             6.15
mo_sin_old_rev_tl_op           3.11
mo_sin_rcnt_rev_tl_op          3.11
mo_sin_rcnt_tl                 3.11
mort_acc                       2.21
mths_since_recent_bc           3.25
mths_since_recent_inq         13.07
num_accts_ever_120_pd          3.11
num_actv_bc_tl                 3.11
num_actv_rev_tl                3.11
num_bc_sats                    2.59
num_bc_tl                      3.11
num_il_tl                   

In [48]:
als = missing[missing != 0].index.tolist()
for i in als:
    if len(df1[i].unique())<20:
           print("Unique " ,i,":", len(df1[i].unique()))

Unique  emp_length : 12
Unique  collections_12_mths_ex_med : 17
Unique  chargeoff_within_12_mths : 12
Unique  num_tl_120dpd_2m : 8
Unique  num_tl_30dpd : 6
Unique  pub_rec_bankruptcies : 13


In [71]:
for i in als:
    if len(df1[i].unique())<20:
        print(i, exc.loc[i])
        print("Unique " ,i,":", df1[i].unique(),"\n\n")

emp_length Description    Employment length in years. Possible values are between 0 and 10 where 0 means less than one yea...
Name: emp_length, dtype: object
Unique  emp_length : ['10+ years' '3 years' '4 years' '6 years' '1 year' '7 years' '8 years'
 '5 years' '2 years' '9 years' '< 1 year' nan] 


collections_12_mths_ex_med Description    Number of collections in 12 months excluding medical collections
Name: collections_12_mths_ex_med, dtype: object
Unique  collections_12_mths_ex_med : [ 0.  1.  2.  3.  4.  5.  7. 10.  6. 14. nan  8. 20.  9. 11. 12. 16.] 


chargeoff_within_12_mths Description    Number of charge-offs within 12 months
Name: chargeoff_within_12_mths, dtype: object
Unique  chargeoff_within_12_mths : [ 0.  1.  2.  3.  6.  7.  4.  5.  9. 10. nan  8.] 


num_tl_120dpd_2m Description    Number of accounts currently 120 days past due (updated in past 2 months)
Name: num_tl_120dpd_2m, dtype: object
Unique  num_tl_120dpd_2m : [ 0. nan  1.  2.  6.  3.  7.  4.] 


num_tl_30dpd 

In [76]:
for i in als:
    if len(df1[i].unique())>=20:
        try:
            print(i, exc.loc[i], "\n\n")
        except:
            print(i, "Not found", "\n\n")

emp_title Description    The job title supplied by the Borrower when applying for the loan.*
Name: emp_title, dtype: object 


title Description    The loan title provided by the borrower
Name: title, dtype: object 


dti Description    A ratio calculated using the borrower’s total monthly debt payments on the total debt obligation...
Name: dti, dtype: object 


revol_util Description    Revolving line utilization rate, or the amount of credit the borrower is using relative to all a...
Name: revol_util, dtype: object 


last_pymnt_d Description    Last month payment was received
Name: last_pymnt_d, dtype: object 


tot_coll_amt Description    Total collection amounts ever owed
Name: tot_coll_amt, dtype: object 


tot_cur_bal Description    Total current balance of all accounts
Name: tot_cur_bal, dtype: object 


total_rev_hi_lim Not found 


acc_open_past_24mths Description    Number of trades opened in past 24 months.
Name: acc_open_past_24mths, dtype: object 


avg_cur_bal Descriptio

In [75]:
df1.isnull().sum()

id                                 0
loan_amnt                         33
funded_amnt                       33
funded_amnt_inv                   33
term                              33
int_rate                          33
installment                       33
grade                             33
sub_grade                         33
emp_title                     167002
emp_length                    146940
home_ownership                    33
annual_inc                        37
verification_status               33
issue_d                           33
loan_status                       33
pymnt_plan                        33
url                               33
purpose                           33
title                          23358
zip_code                          34
addr_state                        33
dti                             1744
delinq_2yrs                       62
earliest_cr_line                  62
fico_range_low                    33
fico_range_high                   33
i

The columns emp_title, emp_length have 7.39% and 6.50% missing value. These columns have information about the customer/borrower like their job title and their employment length in years. Let's remove the rows with nan/blank values for these variables. Other variables have very less data missing compared to total data size. So, let's remove the rows with the missing features.

Now, we have now reduced the data set to have 1652452 loan records and 93 attributes with no missing values. 

In [77]:
loan = df1.dropna()
loan.shape

(1652452, 93)

Let's now summarise and save the data to a new clean data file

In [ ]:
loan.to_csv('clean_loan.csv', encoding='utf-8', index=False)

> # 